In [1]:
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import seaborn as sns

style.use('ggplot')
%matplotlib inline

In [2]:
import numpy
import math 
import os 
import random
from datetime import datetime
random.seed(datetime.now())

import cntk as C

In [3]:
# Select the right target device when this notebook is being tested:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))

In [4]:
class CardSet:
    def __init__(self):
        self.cards = []
        self.num_under_sea = 16
        self.reset()
        
    def reset(self):
        self.cards = []
        self.num_under_sea = 16
        for i in range(144):
            self.cards.append( self.cardIDcompress(i) )
        self.shuffle() 
        
    def cardIDcompress(self, idx):
        if idx < 0:
            print ("id should be larger than or equal to 0")
        if idx < 136:
            return int(idx/4)
        elif idx < 144:
            return int(idx-136 + 136/4)
        else:
            print ("id should be less than 144")
        return -1 # error
    
    def draw(self, direction="front"):
        if self.canDraw() == False:
            print("can't draw anymore")
            return -1
            
        if direction == "front":    
            card = self.cards[0]
            self.cards = self.cards[1:]
        elif direction == "back":
            card = self.cards[len(self.cards)-1]
            self.cards = self.cards[:-1]
        else:
            print("direction should be (front) or (back)")
            return -1
            
        return card
    
    def canDraw(self):
        if self.cardRemain() <= self.num_under_sea:
            return False
        return True
    
    def cardRemain(self):
        return len(self.cards)
    
    def shuffle(self):
        cardN = self.cardRemain()
        for i in range(cardN):
            idx = random.randint(i, cardN-1)
            temp = self.cards[i]
            self.cards[i] = self.cards[idx]
            self.cards[idx] = temp

# cardSet = CardSet()
# print(cardSet.cards)

In [5]:
def isEatable(card):
    if card in range(27):
        return True
    else:
        return False

In [51]:
# M1~M9, C1~C9, L1~L9, EW,WW,SW,NW, R,G,W, F1~8
# 9,    18,    27    24, 

def cardID2Str(idx):
    if idx < 0:
        print ("id should be larger than or equal to 0")
        return -1
    if idx < 27:
        card_num = int(idx%9) + 1
        card_type = int(idx/9)
        card_type_list = ["M", "C", "L"]
        return card_type_list[card_type] + str(card_num)
    elif idx < 34:
        card_type = int(idx-27)
        card_type_list = ["EW","WW","SW","NW", "RC","GF","WB"]
        return card_type_list[card_type]
    elif idx < 42:
        card_num = int((idx-34))+1
        return "F"+ str(card_num)
    else:
        print ("id should be less than 144")
        return -1
        
    return -1

def cardSet2Str(cards):
    cards_str = []
    for i in range(len(cards)):
        cards_str.append(cardID2Str(cards[i]))
    return cards_str

def cardStr2ID(s):
    MCL_offset = 0
    W_offset = 27
    CFB_offset = 31
    F_offset = 34
    if s[0] in ['M', 'C', 'L']:
        return ['M', 'C', 'L'].index(s[0])*9 + (int(s[1]) -1)
    if s[1] == 'W':
        d = {'E':0, 'W':1, 'S':2, 'N':3}
        return W_offset + d[s[0]]
    if s in ['RC', 'GF', 'WB']:
        return CFB_offset + ['RC', 'GF', 'WB'].index(s)
    if s[0] == 'F':
        return F_offset + (int(s[1]) -1)
    
    print("Error cardStr2ID:  ", s)
    return -1

def cardSet2ID(cards):
    cards_ID = []
    for i in range(len(cards)):
        cards_ID.append(cardStr2ID(cards[i]))
    return cards_ID

cards = range(42)
print(cardSet2Str(cards))
if not (cardSet2ID(cardSet2Str(cards)) == list(cards)): 
    print ("error 1")
if not (cardSet2Str(cardSet2ID(cardSet2Str(cards))) == cardSet2Str(cards)):
    print ("error 2")

['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'EW', 'WW', 'SW', 'NW', 'RC', 'GF', 'WB', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8']


In [38]:
s = '1'
int(s)

1

In [7]:
def isFlower(card):
    return cardID2Str(card)[0] == 'F'

def showFlowers(cards):
    flowers = []
    flower_index_list = []
    for index, card in enumerate(cards):
        if isFlower(card):
            flowers.append(card)
            flower_index_list.append(index)
    flower_index_list.reverse()
    for index in flower_index_list:
        cards.pop(index)
    return flowers

c = [1,3,3,39,41]
f = [40]
f.extend(showFlowers(c))
if not ( f == [39,40,41] and c == [1,3,3]):
    print("err: c=",c, ", f=", f)

err: c= [1, 3, 3] , f= [40, 39, 41]


In [12]:
# M1~M9, C1~C9, L1~L9, EW,WW,SW,NW, R,G,W, F1~8
# 36,    72,    108    136, 
# action = win + play(34) + eat(3) + pon + gan(34) + pass
    # eat0 = eat downward, ex: eat 5 with 67
    # eat1 = eat middle,   ex: eat 5 with 46
    # eat2 = eat upward    ex: eat 5 with 34

# Feature_From = [0,1,2], 0=self, 1=prevPlayer, 2=from other players
TABLE_STATE_SIZE = 34      # 34 types of card and their quentities (no flowers)
# TABLE_STATE_SIZE = 64    # beacause there is at most 64 card on the table
HAND_STATE_SIZE = 34
OPEN_STATE_SIZE = 42       # 42 = all types including flowers# state = from + in_card + hand + open + (table)
STATE_COUNT = 1 + 1 + HAND_STATE_SIZE + OPEN_STATE_SIZE + TABLE_STATE_SIZE
ACTION_COUNT = 1 + 34 + 3 + 1 + 34 + 1

# state = from + in_card + hand + open + (table)
STATE_COUNT = 1 + 1 + HAND_STATE_SIZE + OPEN_STATE_SIZE + TABLE_STATE_SIZE
ACTION_COUNT = 1 + 34 + 3 + 1 + 34 + 1

In [13]:

H = 64 # hidden layer size

class Brain:
    def __init__(self):
        self.params = {}
        self.model, self.trainer, self.loss = self._create()
        
    def _create(self):
        observation = C.sequence.input_variable(STATE_COUNT, np.float32, name="s")
        q_target = C.sequence.input_variable(ACTION_COUNT, np.float32, name="q")
        
        # Following a style similar to Keras
        l1 = C.layers.Dense(H, activation=C.relu)
        l2 = C.layers.Dense(ACTION_COUNT)
        unbound_model = C.layers.Sequential([l1,l2])
        model = unbound_model(observation)
        
        self.params = dict(W1=l1.W, b1=l1.b, W2=l2.W, b2=l2.b)
        
        # loss = 'mse'
        loss = C.reduce_mean(C.square(model - q_target), axis=0)
        meas = C.reduce_mean(C.square(model - q_target), axis=0)
        
        # optimizer
        lr = 0.00025
        lr_schedule = C.learning_rate_schedule(lr, C.UnitType.minibatch)
        learner = C.sgd(model.parameters, lr_schedule, gradient_clipping_threshold_per_sample=10)
        trainer = C.Trainer(model,(loss,meas),learner)
        
        # CNTK: return trainer and loss as well
        return model, trainer, loss

    def train(self, x, y, epoch=1, verbose=0):
        arguments = dict(zip(self.loss.arguments, [x,y]))
        updated, results = self.trainer.train_minibatch(arguments, outputs = [self.loss.output])
        
    def predict(self, s):
        return self.model.eval([s])


In [14]:
class Memory:  # stored as (s, a, r, s_ )
    samples = []
    
    def __init__(self, capacity):
        self.capacity = capacity
        
    def add(self, sample):
        self.samples.append(sample)
        
        if len(self.samples) > self.capacity:
            self.samples.pop(0)
            
    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)

In [26]:
MEMORY_CAPACITY = 100000
BATCH_SIZE = 64

GAMMA = 0.99 # discount factor

MAX_EPSILON = 1
MIN_EPSILON = 0.01 # stay a bit curious even when getting old
LAMBDA = 0.0001    # speed of decay 

class Agent:
    steps = 0
    epsilon = MAX_EPSILON
    
    def __init__(self):
        self.brain = Brain()
        self.memory = Memory(MEMORY_CAPACITY)
        
    def act(self, s, legal_action=[1]*ACTION_COUNT):
        if random.random() < self.epsilon:
            legal_index_list = []
            for index, value in enumerate(legal_action):
                if value == 1:
                    legal_index_list.append(index)
            return random.choice(legal_index_list)
        else:
            # return numpy.argmax(self.brain.predict(s))
            return numpy.argmax( np.array(self.brain.predict(s)) * np.array(legal_action) )
        
    def observe(self, sample): # in (s, a, r, s_) format
        self.memory.add(sample)
        
        # slowly decrease Epsilon based on our experience
        self.steps += 1
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)
        
    def replay(self):
        batch = self.memory.sample(BATCH_SIZE)
        batchLen = len(batch)
        
        no_state = numpy.zeros(STATE_COUNT)
        
        # CNTK: explicity setting to float32
        states = numpy.array([ o[0] for o in batch ], dtype = np.float32)
        states_ = numpy.array([(no_state if o[3] is None else o[3]) for o in batch ] , dtype = np.float32)
        
        p = self.brain.predict(states)
        p_ = self.brain.predict(states_)
        
        # CNTK: explicitly setting to float32
        x = numpy.zeros((batchLen, STATE_COUNT)).astype(np.float32)
        y = numpy.zeros((batchLen, ACTION_COUNT)).astype(np.float32)
        
        for i in range(batchLen):
            s, a, r, s_ = batch[i]
            
            # CNTK:[0] beacuse of sequence dimension
            t = p[0][i]
            if s_ is None:
                t[a] = r
            else:
                t[a] = r + GAMMA * numpy.amax(p_[0][i])
                
            x[i] = s
            y[i] = t
        
        self.brain.train(x,y)

In [16]:
def actionID2Str(action):
    actionStr = ""
    # action = win + play(34) + eat(3) + pon + gan(34)
    # action : win:0, play:1~34, eat:35~37, pon:38, gan:39~72, pass:73
        # eat0 = eat downward, ex: eat 5 with 67
        # eat1 = eat middle,   ex: eat 5 with 46
        # eat2 = eat upward    ex: eat 5 with 34
    if action < 0:
        # error
        actionStr = "Error, action should >= 0"
    elif action == 0:
        # win check
        actionStr = "win"
    elif action <= 34:
        # play card with responding index
        cardID = action-1
        actionStr = "play: " + cardID2Str(cardID)

    elif action <= 37:
        new_action = action - 35
        # 3 type of eat
        actionStr = "eat (type "+str(new_action)+")"
    elif action == 38:
        # pon
        actionStr = "pon"
    elif action <= 72:
        # gan with responding index
        cardID = action - 39
        actionStr = "gan: " + cardID2Str(cardID)
    elif action == 73:
        # pass the card (if you're curplaying player, you can/cannot pass, bigyo is still comtemplate)
        actionStr = "pass"
    return actionStr

In [17]:
def makeLegalList(card, cards, frm=0):
    legal_win = [0]
    legal_play = [0]*34
    legal_eat = [0]*3
    legal_pon = [0]
    legal_gan = [0]*34
    legal_pass = [1]
    if checkWin(card, cards):
        legal_win = [1]
    if frm == 0: 
        for val in cards:
            legal_play[val] = 1
        if not card == -1:     #this means player doen't draw card, EX: in the playing card state after pon
            legal_play[card] = 1
        legal_gan = checkLegalGan(card, cards)
        legal_pass = [0]
    if frm == 1:
        # check eat
        legal_eat = checkEat(card, cards)
        # check pon
        legal_pon = checkPon(card, cards)
        pass
    if frm == 2:
        # check pon
        legal_pon = checkPon(card, cards)
        # check gan
        legal_gan[card] = 1 if checkGanFromOther(card, cards) else 0
        pass

    # action = win + play(34) + eat(3) + pon + gan(34) + pass
    legal_action_list = legal_win + legal_play + legal_eat + legal_pon + legal_gan + legal_pass
    return legal_action_list

def checkEat(card, cards):
    #check the card is MCL
    # M1~M9, C1~C9, L1~L9, 
    # 9,    18,    27
    legal_eat = [0,0,0]
    if not card in range(27):
        return legal_eat
    if card%9 in range(7):
        if card+1 in cards and card+2 in cards:
            legal_eat[0] = 1
    if card%9 in range(1, 8):
        if card+1 in cards and card-1 in cards:
            legal_eat[1] = 1
    if card%9 in range(2,9):
        if card-1 in cards and card-2 in cards:
            legal_eat[2] = 1
    return legal_eat

def checkPon(card, cards):
    count = 0
    for val in cards:
        if val == card:
            count += 1
    return [1] if (count>=2) else [0]

def checkGanFromOther(card, cards):
    # cards >= 3
    count = 0
    for val in cards:
        if val == card:
            count += 1
    return (count>=3)

def checkLegalGan(card, cards):
    legal_gan = [0]*34
    counts = [0]*34
    counts[card] += 1
    for val in cards:
        counts[val] += 1
    for i in range(len(counts)):
        if counts[i] == 4:
            legal_gan[i] = 1
        elif counts[i] > 4:
            "checkLegalGan error: how is this possible?"
    return legal_gan

if not checkLegalGan(1, [1,1,1,2,2,2,2]) == [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
    print("function checkLegalGan error")
    
if not checkGanFromOther(1, [1,1,1,2,2,2,2]):
    print("function checkGanFromOther error")
    
if not ( checkEat(1, [2,3])==[1,0,0] and checkEat(8, [6,7,8,9,10])==[0,0,1] and checkEat(15, [13,14,15,16,17])==[1, 1, 1] and checkEat(27,[26,27,28,29])==[0,0,0]):
    print("function checkEat error")

In [94]:
def checkWin(card, cards):
    if card == -1:
        card_list = cards
    else:
        card_list = [card]+cards
    return checkWinWithEye(card_list)

def checkWinWithEye(cards, eye=0):
    if len(cards) == 0:
        if eye==1:
            return True   # end condition of recursive
        else:
            return False
        
    card = cards[0]
    cards = cards[1:]
#     print("card = {}, cards = {}, eye = {}".format(card, cards, eye))
    
    c_count = 0
    for c in cards:
        if c == card:
            c_count+=1

    if c_count >= 2:
        # split pon and cards, check
        c = cards.copy()
        c.remove(card)
        c.remove(card)
#         print("check pon")
        if checkWinWithEye(c, eye):
            return True
    if c_count >= 1:
        if eye==0:
            c = cards.copy()
            c.remove(card)
#             print("check eye")
            if checkWinWithEye(c, eye+1):
                return True
    if isEatable(card):
        checkList = [[card-2, card-1],[card-1, card+1],[card+1, card+2]]
        for cl in checkList:
            if cl[0]//9 == card//9 and cl[1]//9 == card//9:  # check the same type of card
                if cl[0] in cards and cl[1] in cards:
                    c = cards.copy()
                    c.remove(cl[0])
                    c.remove(cl[1])
#                     print("check eat")
                    if checkWinWithEye(c, eye):
                        return True
    
    
    return False


if not (checkWin(1,[1,1,2,2,28,28,28])):
    print("error 4")

if(checkWin(1,[1,1,2,2,28,29,30])):
    print("error 3")

if(checkWin(1,[1,1])):
    print("error 2")

if not checkWin(1,[1,1,1,2,3,2,3,8,8,8,9,9,9]):
    print("error 1")



In [19]:


class Environment:
    def __init__(self):
        self.cardSet = CardSet()
        self.players = []
        self.cards = []
        self.cards_open = []
        self.table = []
        self.s = [[],[],[],[]]
        self.a = [0,0,0,0]
        self.r = [0,0,0,0]
        self.s_ = [[],[],[],[]]
        self.cur_player_idx = 0
        
        self.reset()
    
    def reset(self):
        # self.table 
        self.table = []
        self.s = [[],[],[],[]]
        self.a = [0,0,0,0]
        self.r = [0,0,0,0]
        self.s_ = [[],[],[],[]]
        
        # reset 4 player's hand card
        self.players = []
        self.cards = []
        self.cards_open = []
        for i in range(4):
            self.players.append(Agent())
            self.cards.append([])
            self.cards_open.append([])
            
        # reset card deck
        self.cardSet.reset()
        
        # deal cards
        for i in range(16):     # card number
            for j in range(4):  # player idx
                #self.cards[j].append(self.cardSet.draw())
                self.drawCard(j)
                
        # showflower
        self.showFlowersAll()
        
        # sorted
        self.sortCardsAll()
            
        # reset pool
        self.pool = []
        
    def drawCard(self, idx):
        self.cards[idx].append(self.cardSet.draw())
        
    
    def receiveFlower(self, idx, card):
        self.cards_open[idx].append(card)
        self.sortCards(idx)
        
    def receiveCard(self, idx, card):
        self.cards[idx].append(card)
        self.sortCards(idx)
        
    def sortCards(self, idx):
        self.cards[idx].sort()
        #self.cards_open[idx].sort()   # i think the open card is better not to sort. or it should be sorted by another function
        
    def sortCardsAll(self):
        for i in range(4):
            self.sortCards(i)
            
    def step(self, idx, card, action): 
        isWin = False
        isEven = False
        playCard = -1
        # action = win + play(34) + eat(3) + pon + gan(34)
            # eat0 = eat downward, ex: eat 5 with 67
            # eat1 = eat middle,   ex: eat 5 with 46
            # eat2 = eat upward    ex: eat 5 with 34
            
        # put the card in two hand, if it's not -1
        if not card == -1:
            self.receiveCard(idx, card)
            
        if action < 0:
            # error
            pass
        elif action == 0:
            # win check
            isWin = checkWin(card, self.cards[idx])
            if isWin == False:
                print("checkWin error !!! in Env.step")
            pass
        elif action <= 34:
            # play card with responding index and remove the card from hand
            playCard = action-1
            self.cards[idx].remove(playCard)
            self.table.append(playCard)
        elif action <= 37:
            new_action = action - 35
            # todo: maybe i can put the eaten card in the center in the future
            remove_cards_list = [card, card+1, card+2]  # ex:  eat 5 with 5,6,7 (type 0), with 4,6 (t 1), with 3,4 (t2)
            for c in remove_cards_list:
                c -= new_action # open card smaller with eating type
                self.cards[idx].remove(c)
                self.cards_open[idx].append(c)
            
            # 3 type of eat
        elif action == 38:
            # pon
            ponCard = card
            for i in range(3):
                self.cards[idx].remove(ponCard)
                self.cards_open[idx].append(ponCard)
            pass
        elif action <= 72:
            new_action = action - 39
            ganCard = new_action
            for i in range(4):
                if not ganCard in self.cards[idx]:
                    print ("gan error in env.step, want to gan {}".format(ganCard) )
                    print("cards now = ", self.cards[idx])
                self.cards[idx].remove(ganCard)
                self.cards_open[idx].append(ganCard)
            # gan with responding index
        elif action == 73:
            # pass the card (if you're curplaying player, you can/cannot pass, bigyo is still comtemplate)
            pass
        
        isEven = self.isEven()
        return isWin, isEven, playCard
    
    def isEven(self):
        if self.cardSet.canDraw() == False:
            return True
        return False
    
    def nextPlayer(self):
        self.cur_player_idx += 1
        self.cur_player_idx %= 4
        
    def showFlowers(self, idx):
        flowers = showFlowers(self.cards[idx])
        self.cards_open[idx].extend(flowers)
        for i in range(len(flowers)):
            self.drawCard(idx)
        return len(flowers)
        
    def showFlowersAll(self):
        finish_flag = False
        while finish_flag == False:
            finish_flag = True
            for i in range(4):
                flower_num = self.showFlowers(i)
                if flower_num > 0 :
                    finish_flag = False
    
    def makeState(self, idx, card, featureFrom=0):
        # state = from + in_card + hand + open + (table)
        hand = self.encodeCards(self.cards[idx])
        cards_open = self.encodeCards(self.cards_open[idx], 42)  # cards which are opened
        table = self.encodeCards(self.table)
        
        state = [featureFrom] + [card] + hand + cards_open + table 
        if not len(state) == STATE_COUNT:
            print("ERROR: state len wrong !! len(state) = %d, STATE_COUNT = %d"%(len(state), STATE_COUNT))
        
        return state
    
    def encodeCards(self, cards, num=34):
        enc = [0] * num
        for card in cards:
            if not card in range(num):
                print("encodeCards error: card index {} out of range".format(card))
                print("card = ", cardID2Str(card))
                # for example flower in hand will cause error
            enc[card] += 1
        return enc

In [20]:
def render(env, level=1):
    env.sortCardsAll()
    printGameDetail("Deck: \n"+str(cardSet2Str(env.cardSet.cards)), level)
    printGameDetail("Table: \n"+str(cardSet2Str(env.table)), level)
    for i in range(4):
        printGameDetail("P%d: \n\t"%(i)+ str(cardSet2Str(env.cards[i]))+ "\n\t"+ str(cardSet2Str(env.cards_open[i])), level)

In [21]:
# action priority
def actionPriority(action):
    # action : win:0, play:1~34, eat:35~37, pon:38, gan:39~72, pass:73
    if action < 0:
        print("actionPriority Error: action should >= 0")
        return
    if action == 0:
        return 3 #win
    if action in range(1,35): 
        return 0
    if action in range(35,38):
        return 1 #eat
    if action == 38:
        return 2 #pon
    if action in range(39,73):
        return 2 #gan
    if action == 73:
        return 0
    

In [22]:
printGameThreshold = 3
def printGameDetail(msg, detail_level=1):
    threshold = printGameThreshold
    if detail_level >= threshold:
        print(msg)

In [23]:
def makeStateAndObserve(env, idx, card):
    env.s[idx] = env.s_[idx]
    env.s_[idx] = env.makeState(idx, card)
    # i want to add observer for the last action because i cannot get the next_state right after action
    if not env.s[idx] == []:
        s = env.s[idx]
        a = env.a[idx]
        r = env.r[idx]
        s_ = env.s_[idx]
        env.players[idx].observe((s, a, r, s_))
        env.players[idx].replay()
        # env.player.obs(s,a,r,s_)

In [110]:
def run(env):
    
    turn = 0 # for save, if turn is too large, break.
    save_break_turn = 800
    gan_flag = False
    has_winner = False
    
    while True:
        turn += 10  # for save, if turn is too large, break.
        if turn >= save_break_turn:
            break
        # 1.start
        render(env)

        # while loop unitl sb win or game draw
        idx = env.cur_player_idx
        # 1.1. Draw and act by idx
        draw_dir = "front"
        if gan_flag == True:
            gan_flag = False
            draw_dir = "back"
        

        while True:
            drawCard = env.cardSet.draw(direction="back")
            
            # -1 means run out of card
            if drawCard == -1:
                isEven = True
                break
            
            printGameDetail("P%d => draw: %s."%(idx, cardID2Str(drawCard)))
            
            if isFlower(drawCard):
                env.receiveFlower(idx, drawCard)
                continue
            else:
                # not flower, move on
                break
             
        if drawCard == -1:
            printGameDetail("Draw!",3)
            break

        # self, draw card and react
        from_idx = 0 
        # make state to predict action
        makeStateAndObserve(env, idx, drawCard)
        # cur index player act respond with this s
        env.a[idx] = env.players[idx].act(env.s_[idx] , makeLegalList(drawCard, env.cards[idx],from_idx))
        actionStr = actionID2Str(env.a[idx])
        # 1.2. react with the drawing card
        isWin, isEven, playCard = env.step(idx, drawCard, env.a[idx])
        printGameDetail("P%d => %s."%(idx, actionStr))
        
        #### special method: use continue to go back the look to implement the action gan ###
        if "gan" in actionStr:
            gan_flag = True
            printGameDetail("gan in self's turn\n", -1)
            continue # jump back to the start of while loop, and draw card form the back-side
        
        # check is the game end, if not, continue
        if isWin :
            # set reward
            # winner is curplayer, loser is the others all.
            render(env, level=3)
            printGameDetail("P%d win! Other three players loss!\n"%(idx),3)
            has_winner = True
            break
        if isEven :
            printGameDetail("Draw!",3)
            break

        while True and (isWin == False):
            turn += 1  # for save, if turn is too large, break.
            if turn >= save_break_turn:
                break
            
            # 1.3. idx play one card, other player react
            # 1.3.0: init priority and index
            react_priority = 0
            react_player = -1  # -1 means nobody react
            react_action = -1
            for i_circular in range(1,4):
                i = (idx+i_circular) % 4
                if i_circular == 1:
                    from_idx = 1 # pre player
                elif i_circular in [2,3]:
                    from_idx = 2 # other players

                # 1.3.1. ask three other players if they want to do something
                # make state to predict action
                makeStateAndObserve(env, i, playCard)
                # ask player what they would do with this state
                tmp_act = env.players[i].act(env.s_[i] , makeLegalList(playCard, env.cards[i],from_idx))
                
                #Debug 8.9.2017
                if tmp_act == 0:
                    printGameDetail("playCard:"+str(playCard)+", cards:"+str(env.cards[i])  ,3)
                    printGameDetail(str(makeLegalList(playCard, env.cards[i],from_idx)),3)
                    printGameDetail("env.act = "+str(env.players[i].act(env.s_[i] , makeLegalList(playCard, env.cards[i],from_idx))),3)
                    test_act  =[]
                    for j in range(20):
                        test_act.append(env.players[i].act(env.s_[i] , makeLegalList(playCard, env.cards[i],from_idx)))
                    printGameDetail("test_act = "+str(test_act),3)
                    printGameDetail("tmp_act = "+str(tmp_act), 3)
                
                actionStr = actionID2Str(tmp_act)
                printGameDetail("\tP%d want to: %s."%(i, actionStr))
                # updata priority and check if the player got the priority
                prior = actionPriority(tmp_act)
                if prior > react_priority:
                    react_priority = prior
                    react_player = i
                    react_action = tmp_act

            # 1.3.2. arrange priority and select the player to do his/her action 
            if react_priority == 0:
                break
            else:
                # pickup the card from table
                if playCard == env.table[-1]:
                    env.table.pop(-1)
                else:
                    print("error the last card of table different form playCard")
                
                # todo: reset curplayer
                env.cur_player_idx = react_player
                idx = env.cur_player_idx
                printGameDetail("Priority != 0")
                env.a[idx] = react_action # save the reacting action to the player
                actionStr = actionID2Str(env.a[idx])
                # play one card after eat or pon or gan other player's card
                isWin, isEven, playCard = env.step(idx, playCard, env.a[idx])
                printGameDetail("P%d => %s."%(idx, actionStr))
                
                # todo: if gan here, jump back to draw
                
                # check if the game end
                if isWin:
                    # set reward
                    # winner is curplayer, loser is the others all.
                    render(env, level=3)
                    printGameDetail("P%d win! Other three players loss!\n"%(idx),3)
                    has_winner = True
                    break
                
                # make state to predict action and observe
                makeStateAndObserve(env, idx, -1)
                # cur index player act respond with this s
                env.a[idx] = env.players[idx].act(env.s_[idx] , makeLegalList(-1, env.cards[idx], 0))
                actionStr = actionID2Str(env.a[idx])
                # 1.2. react with the drawing card
                isWin, isEven, playCard = env.step(idx, -1, env.a[idx])
                printGameDetail("P%d => %s."%(idx, actionStr))

        if isWin:
            render(env, level=3)
            printGameDetail("P%d win! Other three players loss!\n"%(idx),3)
            has_winner = True
            break

        # 1.4. switch to next player
        env.nextPlayer()
        printGameDetail("nextPlayer\n")
        # return observation, reward, done, info
    
    # can do somethin here after game loop
    printGameDetail("Game: End!\n",3)
    
    return has_winner

In [111]:
env = Environment()
run(env)

Draw!

Game: End!



False

In [112]:
printGameThreshold = 3
for i in range(20):
    env = Environment()
    has_winner = run(env)
    if has_winner:
        break

Draw!

Game: End!

can't draw anymore
Draw!
Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

can't draw anymore
Draw!
Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

Draw!

Game: End!

playCard:12, cards:[1, 4, 5, 5, 9, 12, 15, 17, 17, 19, 32, 32, 33]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
env.act = 73
test_act = [73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73]
tmp_act = 0
checkWin error !!! in Env.step
Draw!

Game: End!

playCard:32, cards:[8, 8, 10, 14, 18, 27, 27]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [114]:
def callBreak():
    break

for i in range(5):
    callBreak()
    print(i)

SyntaxError: 'break' outside loop (<ipython-input-114-270240fbd602>, line 2)

In [30]:
printGameThreshold = 4
# print: 3:gameend, 1:gamedetail

win_count = 0
for i in range(100):
    env = Environment()
    has_winner = run(env)
    if has_winner:
        win_count+=1
        
print("win_count = ", win_count)

can't draw anymore
checkWin error !!! in Env.step
checkWin error !!! in Env.step
checkWin error !!! in Env.step
checkWin error !!! in Env.step
checkWin error !!! in Env.step
checkWin error !!! in Env.step
checkWin error !!! in Env.step
gan error in env.step, want to gan 33
cards now =  [0, 0, 1, 7, 8, 15, 26, 26]


ValueError: list.remove(x): x not in list

In [ ]:
# todo : make other play take acts
# [v]todo : check eat,
# [v]todo : act eat
# [v]deal with gan in self's turn
# todo : gan when drawing the card which u pon before
# todo : check win,
# todo : act win
# todo : arrange priority of the other 3 players action and select the top one to do the action

# todo : normalize state parameter

# todo : deal with the condition of empty of deck

# todo : can not win when you play the card in exactly that turn